In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('..')
from src.ham_to_sparse_matrix import *
from src.constants import *
from src.matrix_to_ps import matrix_to_pauli_strings
from src.binary_encodings import *

import sympy as sp
import sys
import scipy.sparse.linalg

In [3]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( -(xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [4]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'dirichlet'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [5]:
ham=0

mass=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)

def potential(n):
    # m*q with m=1
    return -mass*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)


ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
sp.expand(ham)

-0.5 + \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 - \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 + \chi^{\dagger}_{2}*\chi_{2} + p_{0}**2/2 + p_{1}**2/2 + p_{2}**2/2 - q_{0}*q_{2}/8 + 5*q_{0}**2/8 + 3*q_{1}**2/4 - q_{2}*q_{0}/8 + 5*q_{2}**2/8

In [6]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

-1/2 + \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 - \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 + \chi^{\dagger}_{2}*\chi_{2} - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 9*a^{\dagger}_{0}*a_{0}/16 - a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/16 + 5*a^{\dagger}_{1}*a_{1}/8 + a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/16 - a^{\dagger}_{2}*a_{0}/16 + 9*a^{\dagger}_{2}*a_{2}/16 + a^{\dagger}_{2}**2/16 + 9*a_{0}*a^{\dagger}_{0}/16 - a_{0}*a^{\dagger}_{2}/16 - a_{0}*a_{2}/16 + a_{0}**2/16 + 5*a_{1}*a^{\dagger}_{1}/8 + a_{1}**2/8 - a_{2}*a^{\dagger}_{0}/16 + 9*a_{2}*a^{\dagger}_{2}/16 - a_{2}*a_{0}/16 + a_{2}**2/16

In [75]:
hamMat

<512x512 sparse matrix of type '<class 'numpy.complex64'>'
	with 3712 stored elements in Compressed Sparse Row format>

In [7]:
from src.qiskit_utilities import pauli_string_to_op
from qiskit import Aer
from qiskit.utils import QuantumInstance

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes

In [9]:
for cutoff in [2,3,4,5,6]:
    print("--------- Lambda = ", cutoff, " ----------")
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    ens=scipy.sparse.linalg.eigs(hamMat,k=6,sigma=0.0)[0]
    print(ens)


    backend = Aer.get_backend('statevector_simulator')
    qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

    ps=matrix_to_pauli_strings(hamMat,standard_encode)
    op = pauli_string_to_op(ps)

    ansatz = RealAmplitudes((cutoff**N)*(2**N), reps=cutoff)
    vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=qinstance)
    result = vqe.compute_minimum_eigenvalue(op)
    print(result.eigenvalue)


    print("{} & {:.2e} & {:.2e} \\\\".format(cutoff, np.min(ens), result.eigenvalue))

--------- Lambda =  2  ----------
[-0.09974489-3.4929865e-10j -0.09974487-1.3549194e-09j
 -0.09974489-1.9968824e-10j -0.09974477-3.5508591e-10j
  0.15025511-6.3905077e-09j  0.15025507-5.3012017e-09j]
(-0.09497856898149475+0j)
2 & -9.97e-02-3.49e-10j & -9.50e-02+0.00e+00j \\
--------- Lambda =  3  ----------
[-0.05103881+1.3915899e-10j  0.51277786+1.9896474e-08j
  0.5412193 +3.3501140e-09j  0.6670306 -1.1735979e-09j
  0.9489601 -9.0527630e-09j  0.9615373 +6.7003536e-09j]
(0.000608018119855729+0j)
3 & -5.10e-02+1.39e-10j & 6.08e-04+0.00e+00j \\
--------- Lambda =  4  ----------
[1.1747504e-04-4.2573917e-12j 9.0781546e-01-1.2834692e-08j
 9.5637053e-01-3.4315125e-08j 1.0001174e+00+1.0006955e-08j
 1.0327833e+00-5.2724429e-09j 1.2119056e+00-4.2428738e-08j]
(0.22772368176133145+0j)
4 & 1.17e-04-4.26e-12j & 2.28e-01+0.00e+00j \\
--------- Lambda =  5  ----------
[-3.5618120e-04-1.1430382e-11j  9.9956614e-01+2.2216037e-08j
  9.9964273e-01-1.4828714e-08j  1.2239851e+00+5.3151389e-10j
  1.2243878

: 

: 